In [192]:
import os
import sys
import pyedflib
import numpy as np
import scipy
from scipy import signal
import matplotlib.pyplot as plt
from itertools import chain
import visdom
from collections import namedtuple
import pickle
from sklearn.cluster import MiniBatchKMeans


In [2]:
seizureTypes = dict()
seizureTypes['BCKG']=6
seizureTypes['FNSZ']=8
seizureTypes['GNSZ']=9
seizureTypes['SPSZ']=10
seizureTypes['CPSZ']=11
seizureTypes['ABSZ']=12
seizureTypes['TNSZ']=13
seizureTypes['CNSZ']=14
seizureTypes['TCSZ']=15
seizureTypes['ATSZ']=16
seizureTypes['MYSZ']=17


In [109]:
vis = visdom.Visdom()

Setting up a new session...


In [3]:
a = list()
for root,dirs,files in os.walk("tuh_eeg_seizure/v1.5.0/edf"):
    for name in files:
        a.append(os.path.join(root,name))
        #print(os.path.join(root,name))
    
b = [i for i in a if 'tse' in i or '.edf' in i ]

file_list = [i for i in b if not 'tse_bi' in i]
#for f in file_list:
    #print(f)
file_edf = [i for i in file_list if '.edf' in i]
file_tse = [i for i in file_list if '.tse' in i]

In [4]:
bgcount = 0
seizcount = 0
for idx,file in enumerate(file_tse):
    #print(file)
    f = open(file,'r')
    for l in f:
        if not "version" in l and not l=="\n":
            #print(l)
        if "bckg" in l:
            bgcount = bgcount+1
        else:
            seizcount = seizcount+1
    f.close()
print(done)
    

0.0000 20.0000 bckg 1.0000

0.0000 15.9375 bckg 1.0000

15.9375 136.4375 tcsz 1.0000

136.4375 272.0000 bckg 1.0000

0.0000 13.0225 bckg 1.0000

13.0225 143.3225 tcsz 1.0000

143.3225 234.0000 bckg 1.0000

0.0000 45.9300 bckg 1.0000

45.9300 162.2325 tcsz 1.0000

162.2325 224.0000 bckg 1.0000

0.0000 8.8275 bckg 1.0000

8.8275 202.0000 tcsz 1.0000

202.0000 203.0000 bckg 1.0000

0.0000 3.9625 bckg 1.0000

3.9625 115.0600 tcsz 1.0000

115.0600 205.0000 bckg 1.0000

0.0000 1.0000 bckg 1.0000

1.0000 109.9500 tcsz 1.0000

109.9500 175.0000 bckg 1.0000

0.0000 207.9100 bckg 1.0000

207.9100 308.0000 fnsz 1.0000

308.0000 309.0000 bckg 1.0000

0.0000 185.0000 bckg 1.0000

0.0000 295.0000 bckg 1.0000

0.0000 369.0000 bckg 1.0000

0.0000 305.0000 bckg 1.0000

0.0000 314.0000 bckg 1.0000

0.0000 464.0000 bckg 1.0000

0.0000 443.0000 bckg 1.0000

0.0000 22.9850 bckg 1.0000

22.9850 162.3125 cpsz 1.0000

162.3125 334.0000 bckg 1.0000

0.0000 87.3375 bckg 1.0000

87.3375 223.0075 cpsz 1.0000

223

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 938.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 1154.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 290.2520 bckg 1.0000

290.2520 416.2520 gnsz 1.0000

416.2520 482.8720 bckg 1.0000

482.8720 556.9720 gnsz 1.0000

556.9720 647.6600 bckg 1.0000

647.6600 742.9600 gnsz 1.0000

742.9600 879.4600 bckg 1.0000

879.4600 958.4600 gnsz 1.0000

958.4600 1029.5520 bckg 1.0000

1029.5520 1054.8520 gnsz 1.0000

1054.8520 1145.9520 bckg 1.0000

1145.9520 1241.7520 gnsz 1.0000

1241.7520 1280.0000 bckg 1.0000

0.0000 55.0000 bckg 1.0000

0.0000 273.6133 bckg 1.0000

273.6133 344.2227 fnsz 1.0000

344.2227 360.0000 bckg 1.0000

0.0000 279.0000 bckg 1.0000

0.0000 76.6211 bckg 1.0000

76.6211

0.0000 48.0273 bckg 1.0000

48.0273 126.4063 fnsz 1.0000

126.4063 300.0000 bckg 1.0000

0.0000 359.2578 bckg 1.0000

359.2578 434.9492 gnsz 1.0000

434.9492 778.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 19.6758 bckg 1.0000

19.6758 82.0781 gnsz 1.0000

82.0781 601.0000 bckg 1.0000

0.0000 16.0560 bckg 1.0000

16.0560 90.5560 cpsz 1.0000

90.5560 180.6120 bckg 1.0000

180.6120 209.5560 cpsz 1.0000

209.5560 272.3760 bckg 1.0000

272.3760 325.4280 cpsz 1.0000

325.4280 405.0440 bckg 1.0000

405.0440 497.0440 cpsz 1.0000

497.0440 576.3800 bckg 1.0000

576.3800 600.0240 cpsz 1.0000

600.0240 696.4920 bckg 1.0000

696.4920 759.6120 cpsz 1.0000

759.6120 855.0440 bckg 1.0000

855.0440 872.6440 cpsz 1.0000

872.6440 895.9960 bckg 1.0000

895.9960 965.0400 cpsz 1.0000

965.0400 1023.7440 bckg 1.0000

1023.7440 1060.2120 cpsz 1.0000

1060.2120 1110.0440 bckg 1.0000

1110.0440 1187.0600 cpsz 1.0000

1187.0600 1269.6440 bckg 1.0000

1269.6440 1277.0000 cpsz 1.0000

1277.0000 1279.00

0.0000 163.6680 bckg 1.0000

163.6680 190.7656 tnsz 1.0000

190.7656 219.9531 bckg 1.0000

219.9531 231.0547 tnsz 1.0000

231.0547 362.5547 bckg 1.0000

362.5547 373.9531 tnsz 1.0000

373.9531 526.3320 bckg 1.0000

526.3320 548.1328 tnsz 1.0000

548.1328 555.0000 bckg 1.0000

0.0000 114.3516 bckg 1.0000

114.3516 127.8984 tnsz 1.0000

127.8984 205.1641 bckg 1.0000

205.1641 223.7070 tnsz 1.0000

223.7070 262.1055 bckg 1.0000

262.1055 272.0547 tnsz 1.0000

272.0547 300.0000 bckg 1.0000

0.0000 265.8477 bckg 1.0000

265.8477 293.9453 tnsz 1.0000

293.9453 517.0000 bckg 1.0000

0.0000 46.0664 bckg 1.0000

46.0664 79.1680 tnsz 1.0000

79.1680 252.6523 bckg 1.0000

252.6523 270.7539 tnsz 1.0000

270.7539 285.4531 bckg 1.0000

285.4531 295.3711 tnsz 1.0000

295.3711 300.0000 bckg 1.0000

0.0000 142.0664 bckg 1.0000

142.0664 156.6797 tnsz 1.0000

156.6797 403.0742 bckg 1.0000

403.0742 424.4727 tnsz 1.0000

424.4727 601.0000 bckg 1.0000

0.0000 44.3945 bckg 1.0000

44.3945 69.6289 tnsz 1.00


245.4922 260.5469 fnsz 1.0000

260.5469 732.0000 bckg 1.0000

0.0000 231.9023 bckg 1.0000

231.9023 262.4961 fnsz 1.0000

262.4961 598.0000 bckg 1.0000

0.0000 35.1680 bckg 1.0000

35.1680 57.1250 fnsz 1.0000

57.1250 300.0000 bckg 1.0000

0.0000 84.9727 bckg 1.0000

84.9727 116.7109 fnsz 1.0000

116.7109 188.7461 bckg 1.0000

188.7461 239.7773 fnsz 1.0000

239.7773 601.0000 bckg 1.0000

0.0000 280.4492 bckg 1.0000

280.4492 298.9883 fnsz 1.0000

298.9883 300.0000 bckg 1.0000

0.0000 129.7188 bckg 1.0000

129.7188 172.0234 fnsz 1.0000

172.0234 601.0000 bckg 1.0000

0.0000 369.0273 bckg 1.0000

369.0273 394.7578 fnsz 1.0000

394.7578 465.9727 bckg 1.0000

465.9727 492.9805 fnsz 1.0000

492.9805 987.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 198.9727 bckg 1.0000

198.9727 225.3750 fnsz 1.0000

225.3750 601.0000 bckg 1.0000

0.0000 204.6289 bckg 1.0000

204.6289 234.7031 fnsz 1.0000

234.7031 300.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 437.3750 bckg 1.0000

437.

...

0.0000 65.0000 bckg 1.0000

0.0000 11.3125 bckg 1.0000

11.3125 24.6750 fnsz 1.0000

24.6750 105.0000 bckg 1.0000

0.0000 38.2700 bckg 1.0000

38.2700 59.7175 fnsz 1.0000

59.7175 118.6300 bckg 1.0000

118.6300 138.1000 fnsz 1.0000

138.1000 145.0000 bckg 1.0000

0.0000 19.6925 bckg 1.0000

19.6925 39.2350 fnsz 1.0000

39.2350 67.7525 bckg 1.0000

67.7525 94.1100 fnsz 1.0000

94.1100 159.5900 bckg 1.0000

159.5900 187.5250 fnsz 1.0000

187.5250 314.0000 bckg 1.0000

0.0000 1593.0000 bckg 1.0000

0.0000 1.0000 bckg 1.0000

1.0000 124.1055 cpsz 1.0000

124.1055 197.5742 bckg 1.0000

197.5742 363.6484 cpsz 1.0000

363.6484 391.9453 bckg 1.0000

391.9453 592.2578 cpsz 1.0000

592.2578 612.0000 bckg 1.0000

0.0000 215.9063 bckg 1.0000

215.9063 321.9648 cpsz 1.0000

321.9648 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 285.0859 bckg 1.0000

285.0859 375.1875 cpsz 

0.0000 150.0000 bckg 1.0000

0.0000 47.0000 bckg 1.0000

0.0000 1.0000 bckg 1.0000

1.0000 299.0000 fnsz 1.0000

299.0000 300.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 343.0000 bckg 1.0000

0.0000 925.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 157.0234 bckg 1.0000

157.0234 299.0000 fnsz 1.0000

299.0000 300.0000 bckg 1.0000

0.0000 604.0000 bckg 1.0000

0.0000 1015.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 894.9063 bckg 1.0000

894.9063 921.0820 gnsz 1.0000

921.0820 1893.0000 bckg 1.0000

0.0000 393.0000 bckg 1.0000

0.0000 82.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 601.0000 bckg 1.0000

0.0000

0.0000 242.0664 bckg 1.0000

242.0664 309.0977 fnsz 1.0000

309.0977 603.0000 bckg 1.0000

0.0000 289.7578 bckg 1.0000

289.7578 317.0078 fnsz 1.0000

317.0078 603.0000 bckg 1.0000

0.0000 276.1563 bckg 1.0000

276.1563 340.8242 fnsz 1.0000

340.8242 602.0000 bckg 1.0000

0.0000 244.8125 bckg 1.0000

244.8125 311.2070 fnsz 1.0000

311.2070 603.0000 bckg 1.0000

0.0000 286.8086 bckg 1.0000

286.8086 360.1953 fnsz 1.0000

360.1953 603.0000 bckg 1.0000

0.0000 260.0898 bckg 1.0000

260.0898 337.0195 fnsz 1.0000

337.0195 604.0000 bckg 1.0000

0.0000 274.8945 bckg 1.0000

274.8945 327.0273 fnsz 1.0000

327.0273 602.0000 bckg 1.0000

0.0000 276.9844 bckg 1.0000

276.9844 333.7500 fnsz 1.0000

333.7500 603.0000 bckg 1.0000

0.0000 258.6289 bckg 1.0000

258.6289 316.9414 fnsz 1.0000

316.9414 741.9922 bckg 1.0000

741.9922 788.7695 fnsz 1.0000

788.7695 1083.0000 bckg 1.0000

0.0000 263.1758 bckg 1.0000

263.1758 324.0313 fnsz 1.0000

324.0313 735.9766 bckg 1.0000

735.9766 799.1016 fnsz 1.00

0.0000 601.0000 bckg 1.0000

0.0000 300.0000 bckg 1.0000

0.0000 300.5586 bckg 1.0000

300.5586 385.0586 fnsz 1.0000

385.0586 691.0000 bckg 1.0000

0.0000 302.4102 bckg 1.0000

302.4102 393.4883 fnsz 1.0000

393.4883 693.0000 bckg 1.0000



NameError: name 'done' is not defined

In [25]:
signal_labels=[]
for i in np.arange(len(file_edf)):
    with pyedflib.EdfReader(file_edf[i]) as f:    
        signal_labels.append(f.getSignalLabels())     
print(signal_labels[0][0])

EEG FP1-REF


In [43]:
# read one signal buffer
sigbufs=[]
with pyedflib.EdfReader(file_edf[0]) as f:
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
        sigbufs[i, :] = f.readSignal(i)
sigbufs = sigbufs.transpose();

In [49]:
channels = [ind for ind, i in enumerate(signal_labels) if 'EEG' in signal_labels[ind]]
eeg_labels = [signal_labels[i] for i in channels]
print(eeg_labels)

['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG EKG-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG T1-REF', 'EEG T2-REF', 'EEG SP1-REF', 'EEG SP2-REF', 'EEG LUC-REF', 'EEG RLC-REF', 'EEG RESP1-REF', 'EEG RESP2-REF', 'EEG 31-REF', 'EEG 32-REF']


In [40]:
print(type(signal_labels))
# = [i for i in file_list if '.tse' in i]


#list2 = [x for ind, x in enumerate(list1) if 4 > ind > 0]


c = [ind for ind, i in enumerate(signal_labels) if 'EEG' in signal_labels[ind]]
print(c)

<class 'list'>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [194]:
# construct time series of seizure classification
data_train = list()
classification = list()

for idx,file in enumerate(file_tse):    
    #idx = 1 # for debug
    file=file_tse[idx]
    with open(file,'r') as tse:        
        with pyedflib.EdfReader(file_edf[idx]) as edf:            
            # get all EEG channels
            signal_labels = edf.getSignalLabels()                    
            channels = [ind for ind, i in enumerate(signal_labels) if 'EEG' in signal_labels[ind]]
            eeg_labels = [signal_labels[a] for a in channels]
            # get number of samples in an EEG channel
            nSamples = edf.getNSamples()[channels[0]]            
            data = np.zeros((nSamples, len(channels)))
            # read in signals
            n = edf.signals_in_file
            d_idx = 0
            for i in channels:            
                data[:,d_idx] = edf.readSignal(i);
                d_idx = d_idx+1

            # make time vector
            fs = edf.getSampleFrequency(channels[channels[0]])
            t = np.arange(nSamples)/float(fs)

            # do butterworth filter on data
            # remove line noise and harmonics
            noise = np.arange(60,fs/2,60)
            for b in noise:            
                sos = signal.butter(5,[b-1,b+1],'bandstop', output='sos', fs = fs);
                data = signal.sosfilt(sos, data, axis=0)

            sos = signal.butter(5,[1],'highpass', output='sos', fs = fs);
            data = signal.sosfilt(sos, data, axis=0)

            # on this file do a stft
            # get channels that are EEG                    
            f_stft, t_stft, z_stft = signal.stft(data, fs=fs,nperseg=fs*1, noverlap=fs*4/5,
                                        return_onesided=True, boundary='zeros', padded=False, axis=0)

            # time series vector that gives the class of eeg at time t that matches the filtered data
            clsf = np.zeros(t_stft.shape)
            for line in tse:            
                if not "version" in line and not line=="\n":                                        
                    #print(line)
                    split = line.split()
                    # construct mask of time points
                    beg = t_stft>=float(split[0])
                    en = t_stft<=float(split[1])
                    mask = np.logical_and(beg,en)
                    clsf[mask] = int(seizureTypes[split[2].upper()])                    
            # after reading the lines and making the flag channel, add it to the per-file list            
            d = dict()
            d['signal'] = (f_stft, t_stft, z_stft)
            d['classification']=clsf            
            d['filename']=file_edf[idx]
            outfile = open(os.path.join('F:/data/',str(idx)+'.pkl'),"wb")
            pickle.dump(d,outfile)
            outfile.close()
            #np.save(os.path.join("./data/",str(idx)),d)
            #data_train.append((f_stft, t_stft, z_stft))
            #classification.append(clsf)
            print(idx, "of", len(file_tse),"data shape is: ", data.shape)


                    
                
print('done')
    

0 of 5610 data shape is:  (8000, 32)
1 of 5610 data shape is:  (108800, 32)
2 of 5610 data shape is:  (93600, 32)
3 of 5610 data shape is:  (89600, 32)
4 of 5610 data shape is:  (81200, 32)
5 of 5610 data shape is:  (82000, 32)
6 of 5610 data shape is:  (70000, 32)
7 of 5610 data shape is:  (123600, 32)
8 of 5610 data shape is:  (74000, 32)
9 of 5610 data shape is:  (118000, 32)
10 of 5610 data shape is:  (147600, 32)
11 of 5610 data shape is:  (122000, 32)
12 of 5610 data shape is:  (125600, 32)
13 of 5610 data shape is:  (185600, 32)
14 of 5610 data shape is:  (177200, 32)
15 of 5610 data shape is:  (133600, 32)
16 of 5610 data shape is:  (141200, 32)
17 of 5610 data shape is:  (84800, 32)
18 of 5610 data shape is:  (140400, 32)
19 of 5610 data shape is:  (116400, 32)
20 of 5610 data shape is:  (216400, 32)
21 of 5610 data shape is:  (200000, 32)
22 of 5610 data shape is:  (331750, 31)
23 of 5610 data shape is:  (153600, 30)
24 of 5610 data shape is:  (337408, 30)
25 of 5610 data sha

205 of 5610 data shape is:  (153856, 30)
206 of 5610 data shape is:  (153856, 30)
207 of 5610 data shape is:  (76800, 30)
208 of 5610 data shape is:  (153856, 30)
209 of 5610 data shape is:  (76800, 30)
210 of 5610 data shape is:  (153856, 30)
211 of 5610 data shape is:  (76800, 30)
212 of 5610 data shape is:  (313088, 26)
213 of 5610 data shape is:  (74752, 26)
214 of 5610 data shape is:  (153856, 26)
215 of 5610 data shape is:  (153856, 26)
216 of 5610 data shape is:  (76800, 26)
217 of 5610 data shape is:  (153856, 26)
218 of 5610 data shape is:  (153856, 26)
219 of 5610 data shape is:  (76800, 26)
220 of 5610 data shape is:  (153856, 26)
221 of 5610 data shape is:  (76800, 26)
222 of 5610 data shape is:  (153856, 26)
223 of 5610 data shape is:  (153856, 26)
224 of 5610 data shape is:  (151552, 26)
225 of 5610 data shape is:  (76800, 26)
226 of 5610 data shape is:  (153856, 26)
227 of 5610 data shape is:  (66816, 26)
228 of 5610 data shape is:  (401750, 26)
229 of 5610 data shape is

407 of 5610 data shape is:  (142080, 30)
408 of 5610 data shape is:  (76800, 30)
409 of 5610 data shape is:  (132352, 30)
410 of 5610 data shape is:  (76800, 30)
411 of 5610 data shape is:  (153856, 30)
412 of 5610 data shape is:  (76800, 30)
413 of 5610 data shape is:  (113664, 30)
414 of 5610 data shape is:  (113408, 30)
415 of 5610 data shape is:  (158208, 30)
416 of 5610 data shape is:  (107776, 30)
417 of 5610 data shape is:  (76800, 30)
418 of 5610 data shape is:  (153856, 30)
419 of 5610 data shape is:  (61696, 30)
420 of 5610 data shape is:  (76800, 30)
421 of 5610 data shape is:  (76800, 30)
422 of 5610 data shape is:  (153856, 30)
423 of 5610 data shape is:  (76800, 30)
424 of 5610 data shape is:  (153856, 30)
425 of 5610 data shape is:  (406250, 26)
426 of 5610 data shape is:  (16000, 26)
427 of 5610 data shape is:  (67584, 30)
428 of 5610 data shape is:  (9216, 30)
429 of 5610 data shape is:  (153856, 30)
430 of 5610 data shape is:  (240384, 30)
431 of 5610 data shape is:  

610 of 5610 data shape is:  (76800, 30)
611 of 5610 data shape is:  (153856, 30)
612 of 5610 data shape is:  (76800, 30)
613 of 5610 data shape is:  (153856, 30)
614 of 5610 data shape is:  (76800, 30)
615 of 5610 data shape is:  (153856, 30)
616 of 5610 data shape is:  (76800, 30)
617 of 5610 data shape is:  (153856, 30)
618 of 5610 data shape is:  (76800, 30)
619 of 5610 data shape is:  (153856, 30)
620 of 5610 data shape is:  (76800, 30)
621 of 5610 data shape is:  (153856, 30)
622 of 5610 data shape is:  (153856, 30)
623 of 5610 data shape is:  (76800, 30)
624 of 5610 data shape is:  (153856, 30)
625 of 5610 data shape is:  (76800, 30)
626 of 5610 data shape is:  (153856, 30)
627 of 5610 data shape is:  (12800, 30)
628 of 5610 data shape is:  (76800, 30)
629 of 5610 data shape is:  (153856, 30)
630 of 5610 data shape is:  (76800, 30)
631 of 5610 data shape is:  (153856, 30)
632 of 5610 data shape is:  (76800, 30)
633 of 5610 data shape is:  (153856, 30)
634 of 5610 data shape is:  

812 of 5610 data shape is:  (153856, 24)
813 of 5610 data shape is:  (76800, 24)
814 of 5610 data shape is:  (206592, 24)
815 of 5610 data shape is:  (76800, 24)
816 of 5610 data shape is:  (153856, 24)
817 of 5610 data shape is:  (153856, 24)
818 of 5610 data shape is:  (76800, 24)
819 of 5610 data shape is:  (153856, 24)
820 of 5610 data shape is:  (153856, 24)
821 of 5610 data shape is:  (76800, 24)
822 of 5610 data shape is:  (153856, 24)
823 of 5610 data shape is:  (76800, 24)
824 of 5610 data shape is:  (153856, 24)
825 of 5610 data shape is:  (196096, 24)
826 of 5610 data shape is:  (319232, 24)
827 of 5610 data shape is:  (76800, 24)
828 of 5610 data shape is:  (153856, 24)
829 of 5610 data shape is:  (252416, 24)
830 of 5610 data shape is:  (76800, 24)
831 of 5610 data shape is:  (449280, 24)
832 of 5610 data shape is:  (153856, 24)
833 of 5610 data shape is:  (76800, 24)
834 of 5610 data shape is:  (236288, 24)
835 of 5610 data shape is:  (76800, 24)
836 of 5610 data shape is

...

4762 of 5610 data shape is:  (50400, 32)
4763 of 5610 data shape is:  (43200, 128)
4764 of 5610 data shape is:  (12400, 128)
4765 of 5610 data shape is:  (28400, 128)
4766 of 5610 data shape is:  (59200, 128)
4767 of 5610 data shape is:  (14000, 128)
4768 of 5610 data shape is:  (81200, 128)
4769 of 5610 data shape is:  (122000, 32)
4770 of 5610 data shape is:  (58000, 32)
4771 of 5610 data shape is:  (49600, 32)
4772 of 5610 data shape is:  (122000, 32)
4773 of 5610 data shape is:  (74000, 32)
4774 of 5610 data shape is:  (74000, 32)
4775 of 5610 data shape is:  (49600, 32)
4776 of 5610 data shape is:  (77600, 32)
4777 of 5610 data shape is:  (50000, 128)
4778 of 5610 data shape is:  (98000, 128)
4779 of 5610 data shape is:  (74000, 128)
4780 of 5610 data shape is:  (78000, 128)
4781 of 5610 data shape is:  (146000, 128)
4782 of 5610 data shape is:  (145600, 128)
4783 of 5610 data shape is:  (108400, 32)
4784 of 5610 data shape is:  (203600, 32)
4785 of 5610 data shape is:  (104000, 1

4958 of 5610 data shape is:  (76800, 28)
4959 of 5610 data shape is:  (153856, 28)
4960 of 5610 data shape is:  (205056, 28)
4961 of 5610 data shape is:  (153856, 28)
4962 of 5610 data shape is:  (177920, 28)
4963 of 5610 data shape is:  (76800, 28)
4964 of 5610 data shape is:  (153856, 28)
4965 of 5610 data shape is:  (511232, 28)
4966 of 5610 data shape is:  (76800, 28)
4967 of 5610 data shape is:  (153856, 28)
4968 of 5610 data shape is:  (153856, 28)
4969 of 5610 data shape is:  (76800, 28)
4970 of 5610 data shape is:  (153856, 28)
4971 of 5610 data shape is:  (76800, 28)
4972 of 5610 data shape is:  (153856, 28)
4973 of 5610 data shape is:  (76800, 28)
4974 of 5610 data shape is:  (153856, 28)
4975 of 5610 data shape is:  (76800, 28)
4976 of 5610 data shape is:  (153856, 28)
4977 of 5610 data shape is:  (76800, 28)
4978 of 5610 data shape is:  (153856, 28)
4979 of 5610 data shape is:  (76800, 28)
4980 of 5610 data shape is:  (155648, 28)
4981 of 5610 data shape is:  (94000, 32)
49

5155 of 5610 data shape is:  (173200, 32)
5156 of 5610 data shape is:  (173600, 32)
5157 of 5610 data shape is:  (177600, 32)
5158 of 5610 data shape is:  (172000, 32)
5159 of 5610 data shape is:  (26000, 32)
5160 of 5610 data shape is:  (169600, 32)
5161 of 5610 data shape is:  (153856, 30)
5162 of 5610 data shape is:  (153856, 30)
5163 of 5610 data shape is:  (153856, 30)
5164 of 5610 data shape is:  (153856, 30)
5165 of 5610 data shape is:  (248400, 32)
5166 of 5610 data shape is:  (112000, 32)
5167 of 5610 data shape is:  (76800, 30)
5168 of 5610 data shape is:  (153856, 30)
5169 of 5610 data shape is:  (153856, 30)
5170 of 5610 data shape is:  (76800, 30)
5171 of 5610 data shape is:  (153856, 30)
5172 of 5610 data shape is:  (153856, 30)
5173 of 5610 data shape is:  (76800, 30)
5174 of 5610 data shape is:  (153856, 30)
5175 of 5610 data shape is:  (188400, 32)
5176 of 5610 data shape is:  (40400, 32)
5177 of 5610 data shape is:  (168400, 32)
5178 of 5610 data shape is:  (120400, 3

5352 of 5610 data shape is:  (153856, 30)
5353 of 5610 data shape is:  (76800, 30)
5354 of 5610 data shape is:  (153856, 30)
5355 of 5610 data shape is:  (76800, 30)
5356 of 5610 data shape is:  (153856, 30)
5357 of 5610 data shape is:  (275200, 22)
5358 of 5610 data shape is:  (153856, 22)
5359 of 5610 data shape is:  (153856, 22)
5360 of 5610 data shape is:  (75000, 22)
5361 of 5610 data shape is:  (153856, 22)
5362 of 5610 data shape is:  (76800, 22)
5363 of 5610 data shape is:  (153856, 22)
5364 of 5610 data shape is:  (76800, 22)
5365 of 5610 data shape is:  (153856, 22)
5366 of 5610 data shape is:  (76800, 22)
5367 of 5610 data shape is:  (153856, 22)
5368 of 5610 data shape is:  (153856, 22)
5369 of 5610 data shape is:  (76800, 22)
5370 of 5610 data shape is:  (153856, 22)
5371 of 5610 data shape is:  (76800, 22)
5372 of 5610 data shape is:  (153856, 22)
5373 of 5610 data shape is:  (76800, 22)
5374 of 5610 data shape is:  (153856, 22)
5375 of 5610 data shape is:  (76800, 22)
53

5549 of 5610 data shape is:  (166656, 28)
5550 of 5610 data shape is:  (76800, 28)
5551 of 5610 data shape is:  (165120, 28)
5552 of 5610 data shape is:  (160512, 28)
5553 of 5610 data shape is:  (161792, 28)
5554 of 5610 data shape is:  (76800, 28)
5555 of 5610 data shape is:  (153856, 28)
5556 of 5610 data shape is:  (153856, 28)
5557 of 5610 data shape is:  (168960, 28)
5558 of 5610 data shape is:  (153856, 28)
5559 of 5610 data shape is:  (76800, 28)
5560 of 5610 data shape is:  (153856, 28)
5561 of 5610 data shape is:  (20480, 30)
5562 of 5610 data shape is:  (56320, 30)
5563 of 5610 data shape is:  (153856, 30)
5564 of 5610 data shape is:  (10496, 30)
5565 of 5610 data shape is:  (153856, 30)
5566 of 5610 data shape is:  (76800, 30)
5567 of 5610 data shape is:  (153856, 30)
5568 of 5610 data shape is:  (76800, 30)
5569 of 5610 data shape is:  (153856, 30)
5570 of 5610 data shape is:  (76800, 30)
5571 of 5610 data shape is:  (153856, 30)
5572 of 5610 data shape is:  (76800, 30)
55

In [260]:
# to open file
#filename = 'F:/data/0.pkl'
#f = open(filename,'rb')
#data = pickle.load(f)
print(signal.shape)
print(d['filename'])
with pyedflib.EdfReader(d['filename']) as edf:
    n = edf.signals_in_file
    signal_labels = edf.getSignalLabels()
    print(n)
    print(signal_labels)
    channels = [ind for ind, i in enumerate(signal_labels) if 'EEG' in signal_labels[ind]]
    eeg_labels = [signal_labels[a] for a in channels]
    
    print(eeg_labels)
    print(data.shape)

(201, 128, 1826)
tuh_eeg_seizure/v1.5.0/edf\train\03_tcp_ar_a\009\00000929\s002_2005_02_28\00000929_s002_t000.edf
129
['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG 20-REF', 'EEG 21-REF', 'EEG 22-REF', 'EEG 23-REF', 'EEG 24-REF', 'EEG 25-REF', 'EEG 26-REF', 'EEG 27-REF', 'EEG 28-REF', 'EEG 29-REF', 'EEG 30-REF', 'EEG 31-REF', 'EEG 32-REF', 'EEG 33-REF', 'EEG 34-REF', 'EEG 35-REF', 'EEG 36-REF', 'EEG 37-REF', 'EEG 38-REF', 'EEG 39-REF', 'EEG 40-REF', 'EEG 41-REF', 'EEG 42-REF', 'EEG 43-REF', 'EEG 44-REF', 'EEG 45-REF', 'EEG 46-REF', 'EEG 47-REF', 'EEG 48-REF', 'EEG 49-REF', 'EEG 50-REF', 'EEG 51-REF', 'EEG 52-REF', 'EEG 53-REF', 'EEG 54-REF', 'EEG 55-REF', 'EEG 56-REF', 'EEG 57-REF', 'EEG 58-REF', 'EEG 59-REF', 'EEG 60-REF', 'EEG 61-REF', 'EEG 62-REF', 'EEG 63-REF

In [258]:
# k means 

kmeans = MiniBatchKMeans(n_clusters = 2, random_state=0, batch_size=100)

# kmeans.partial_fit(data)
a = list()
for root,dirs,files in os.walk("F:/data/"):
    for name in files:
        a.append(os.path.join(root,name))
        #print(os.path.join(root,name))
filelist = a
Y = 32;
for f in np.arange(4001, len(filelist)):
    with open(filelist[f],'rb') as file:
        d = pickle.load(file)
        freqs, t, signal = d['signal']
        #print(np.reshape(signal[:,:,0],(201*32,1)))
        #print(np.reshape(signal,(201*32,101))[:,0])
        #kmeans.partial_fit(np.absolute(signal))
        data = np.zeros((signal.shape[1],signal.shape[2]))
        for i in np.arange(signal.shape[2]):
            n = np.corrcoef(signal[:,:,i].transpose())            
            e = scipy.linalg.eig(np.absolute(n))            
            data[:,i] = np.absolute(e[0])
        data = data.transpose()
        if data.shape[1] < Y:
            z = np.zeros((data.shape[0],Y-data.shape[1]))
            data = np.hstack((data,z))
        if data.shape[1] > Y:
            data = data[:,0:31]
        kmeans.partial_fit(data)
        if f%100 == 0:
            print("saving model after f = ", f)
            pickle.dump(kmeans, open('F:/model.pkl','wb'))
        

saving model after f =  4100


IndexError: too many indices for array

In [124]:
d = np.zeros(t.shape)

for f in [0.5,4,11,19,35,60,120]:
    d = d + np.cos(2*np.pi*t*f)
dfft = scipy.fft.fft(d)
x = np.absolute(dfft)
y = scipy.fft.fftfreq(t.shape[0],1/fs)
vis.line(x,y)
out_f, out_t, out_z = signal.stft(d, fs=fs, window='hann', nperseg=fs*1, noverlap=fs*4/5, \
                          return_onesided=True, boundary='zeros', \
                          padded=True, axis=0)
opts = dict()
opts['xmin'] = 0
opts['xmax'] = 1


vis.heatmap(np.absolute(out_z), opts = opts)
sos = signal.butter(5,[59,61],'bandstop', output='sos', fs = fs);
d = signal.sosfilt(sos, d, axis=0)
sos = signal.butter(5,[1],'highpass', output='sos', fs = fs);
d = signal.sosfilt(sos, d, axis=0)

out_f, out_t, out_z = signal.stft(d, fs=fs, window='hann', nperseg=fs*1, noverlap=fs*4/5, \
                          return_onesided=True, boundary='zeros', \
                          padded=True, axis=0)
vis.heatmap(np.absolute(out_z), opts = opts)

'window_3834230a4ab280'

In [ ]:
# for each file
# do STFT on a window of data to construct x
# take classification for that time point to construct y

In [ ]:
split
split[0]
beg = t>=float(split[0])
en = t<=float(split[1])
mask = np.logical_and(beg,en)
c = mask*int(seizureTypes[split[2].upper()])
plt.plot(t,c)
plt.show()

In [ ]:
int(seizureTypes[split[2].upper()])

In [ ]:
c[mask] = 7

In [ ]:
plt.plot(classification[1])
plt.show()